In [1]:
import os
import sys
import rrdtool
import pickle
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from auserverdata.preparation.rrd import *
from joblib import Parallel,delayed

In [ ]:
results = parse_rrds_for_all_snmp_servers(snmp_path='/home/bshook/Projects/server-log-api/rrd/original/snmp', 
                                          dst=f'/home/bshook/Projects/server-log-api/rrd/parsed/snmp_parsed_data.csv')

In [ ]:
results = parse_rrds_for_all_collectd_servers(collectd_path='/home/bshook/Projects/server-log-api/rrd/original/collectd', 
                                              dst=f'/home/bshook/Projects/server-log-api/rrd/parsed/collectd/parsed_data.csv',
                                              n_jobs=-1)

In [2]:
parsed_rrd = pd.read_csv('/home/bshook/Projects/server-log-api/rrd/parsed/collectd/parsed_data.csv')

In [2]:
rrd_path = '/home/bshook/Projects/server-log-api/rrd'

In [3]:
get_time_series_data_for_collectd_server(rrd_path, collectd_server='gpu1')

component              cpu-0                                               \
rrd                 cpu-idle cpu-interrupt cpu-nice cpu-softirq cpu-steal   
data_source            value         value    value       value     value   
time                                                                        
2019-07-24 07:02:46      NaN           NaN      NaN         NaN       NaN   
2019-07-24 07:02:56      NaN           NaN      NaN         NaN       NaN   
2019-07-24 07:03:06      NaN           NaN      NaN         NaN       NaN   
2019-07-24 07:03:16      NaN           NaN      NaN         NaN       NaN   
2019-07-24 07:03:26      NaN           NaN      NaN         NaN       NaN   
...                      ...           ...      ...         ...       ...   
2023-05-20 04:11:13      NaN           NaN      NaN         NaN       NaN   
2023-05-20 04:11:15      NaN           NaN      NaN         NaN       NaN   
2023-05-20 04:11:16      NaN           NaN      NaN         NaN       NaN   
2023-05-20 04:11:18     99.6           0.0      0.0         0.0       0.0   
2023-05-20 04:11:23      NaN           NaN      NaN         NaN       NaN   

component                                           cpu-1                ...  \
rrd                 cpu-system cpu-user cpu-wait cpu-idle cpu-interrupt  ...   
data_source              value    value    value    value         value  ...   
time                                                                     ...   
2019-07-24 07:02:46        NaN      NaN      NaN      NaN           NaN  ...   
2019-07-24 07:02:56        NaN      NaN      NaN      NaN           NaN  ...   
2019-07-24 07:03:06        NaN      NaN      NaN      NaN           NaN  ...   
2019-07-24 07:03:16        NaN      NaN      NaN      NaN           NaN  ...   
2019-07-24 07:03:26        NaN      NaN      NaN      NaN           NaN  ...   
...                        ...      ...      ...      ...           ...  ...   
2023-05-20 04:11:13        NaN      NaN      NaN      NaN           NaN  ...   
2023-05-20 04:11:15        NaN      NaN      NaN      NaN           NaN  ...   
2023-05-20 04:11:16        NaN      NaN      NaN      NaN           NaN  ...   
2023-05-20 04:11:18       0.08      0.0     0.18     99.9           0.0  ...   
2023-05-20 04:11:23        NaN      NaN      NaN      NaN           NaN  ...   

component           sensors-coretemp-isa-0000                    \
rrd                         temperature-temp6 temperature-temp7   
data_source                             value             value   
time                                                              
2019-07-24 07:02:46                       NaN               NaN   
2019-07-24 07:02:56                       NaN               NaN   
2019-07-24 07:03:06                       NaN               NaN   
2019-07-24 07:03:16                       NaN               NaN   
2019-07-24 07:03:26                       NaN               NaN   
...                                       ...               ...   
2023-05-20 04:11:13                       NaN               NaN   
2023-05-20 04:11:15                      26.5              24.0   
2023-05-20 04:11:16                       NaN               NaN   
2023-05-20 04:11:18                       NaN               NaN   
2023-05-20 04:11:23                       NaN               NaN   

component           sensors-coretemp-isa-0001                    \
rrd                         temperature-temp1 temperature-temp2   
data_source                             value             value   
time                                                              
2019-07-24 07:02:46                       NaN               NaN   
2019-07-24 07:02:56                       NaN               NaN   
2019-07-24 07:03:06                       NaN               NaN   
2019-07-24 07:03:16                       NaN               NaN   
2019-07-24 07:03:26                       NaN               NaN   
...                               

In [5]:
get_available_component_names_for_collectd_server(
    rrd_path,
    collectd_server = 'gpu1',
    parsed_rrd = parsed_rrd
    )

array(['cpu-0', 'cpu-1', 'cpu-10', 'cpu-11', 'cpu-12', 'cpu-13', 'cpu-14',
       'cpu-15', 'cpu-16', 'cpu-17', 'cpu-18', 'cpu-19', 'cpu-2',
       'cpu-20', 'cpu-21', 'cpu-22', 'cpu-23', 'cpu-3', 'cpu-4', 'cpu-5',
       'cpu-6', 'cpu-7', 'cpu-8', 'cpu-9', 'dns', 'interface-eno1',
       'interface-eno2', 'interface-lo', 'ipmi', 'load', 'memory',
       'sensors-coretemp-isa-0000', 'sensors-coretemp-isa-0001',
       'sensors-i350bb-pci-8100'], dtype=object)

In [5]:
get_available_rrd_names_for_collectd_server_component(
    rrd_path,
    collectd_server = 'gpu1',
    component = 'cpu-0',
    parsed_rrd = parsed_rrd
)

['cpu-idle',
 'cpu-interrupt',
 'cpu-nice',
 'cpu-softirq',
 'cpu-steal',
 'cpu-system',
 'cpu-user',
 'cpu-wait']

In [2]:
rrd_path = '/home/bshook/Projects/server-log-api/rrd'
monitor = 'snmp'
servers = ['gpu1']

rrd_df = select_rrd_data_by_server(
                                rrd_path=rrd_path,
                                monitor=monitor,
                                servers=servers,
                                lazy_load=False)


In [5]:
df = get_time_series_data_for_single_rrd(
                                rrd_path = '/home/bshook/Projects/server-log-api/rrd',
                                monitor = 'snmp',
                                server = 'gpu1',
                                rrd = 'processor-hr-196614'
                                )

In [ ]:
get_time_series_data_for_snmp_server(
    rrd_path,
    snmp_server = 'gpu1'
    )